In [1]:
import pandas as pd
import numpy as np
import torch

## Importar base de dados

In [ ]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz


Saved under ml-latest-small (1).zip


x ml-latest-small/
x ml-latest-small/links.csv
x ml-latest-small/tags.csv
x ml-latest-small/ratings.csv
x ml-latest-small/README.txt
x ml-latest-small/movies.csv


In [43]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
tags = pd.read_csv('./dataset/tags_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [44]:
tags

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474
...,...,...,...,...
6269,138301,109487,ambitious,1423178287
6270,138301,1089,nonlinear,1407271610
6271,138301,1089,Quentin Tarantino,1407271608
6272,138301,109487,Self-Indulgent,1423178219


## Mapeamento de ids

In [45]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(movies.movieId.unique())}
df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)
tags['userId'] = tags['userId'].map(map_users)
tags['movieId'] = tags['movieId'].map(map_items)
movies['movieId'] = movies['movieId'].map(map_items)

map_title = {}

for row in df.itertuples():
    map_title[row.movieId] = row.title

## Divisão da base em treino e teste

In [46]:
from sklearn.model_selection import train_test_split
# REMOVER DEPOIS
#df = df.iloc[:20000]
train, test = train_test_split(df, test_size=.2, random_state=2)

## Modelo EASE

In [ ]:
class EASE:
  def __init__(self, train: pd.DataFrame, lambda_: float):
    """
    Initialize the EASE (Embarrassingly Shallow Autoencoder) model for collaborative filtering.
    
    Args:
        train (pd.DataFrame): Training data containing user-item interactions
                            Expected columns: ['userId', 'movieId']
        lambda_ (float): Regularization parameter to prevent overfitting
    
    The EASE algorithm creates a self-expressiveness matrix that can reconstruct
    the original user-item interaction matrix through a closed-form solution.
    """
    # Convert user-item interactions to sparse tensor format
    self.indices = torch.LongTensor(train[['userId', 'movieId']].values)
    self.values = torch.ones(self.indices.shape[0])
    self.sparse = torch.sparse.FloatTensor(self.indices.t(), self.values)
    
    # Convert sparse matrix to dense and compute Gram matrix
    # G = X^T * X where X is the user-item interaction matrix
    G = self.sparse.to_dense().t() @ self.sparse.to_dense()
    
    # Add regularization term (lambda * I) to the diagonal
    # This helps prevent overfitting and ensures matrix invertibility
    G += torch.eye(G.shape[0])*lambda_
    
    # Compute the closed-form solution
    # P = (X^T * X + λI)^(-1)
    P = G.inverse()
    
    # Calculate the self-expressiveness matrix B
    # Normalize by the negative diagonal entries
    self.B = P / (-1*P.diag())
    
    # Add identity matrix to prevent self-recommendation
    # Final B matrix will have zeros on the diagonal
    self.B = self.B + torch.eye(self.B.shape[0])
  
  def predict_all(self, pred_df: pd.DataFrame, k: int = 5, remove_owned: bool = True):
    """
    Generate top-k recommendations for each user using the EASE model.
    
    Args:
        pred_df (pd.DataFrame): DataFrame containing users for whom to generate predictions
                              Must contain a 'userId' column
        k (int): Number of recommendations to generate per user
        remove_owned (bool): If True, penalizes items the user has already interacted with
                           to avoid recommending them again
    
    Returns:
        pd.DataFrame: DataFrame of users + their predictions in sorted order
    """
    # Keep only unique users from prediction DataFrame
    pred_df = pred_df[['userId']].drop_duplicates()

    # Store predictions for all users
    _output_preds = []
    
    # Convert sparse user-item interaction matrix to dense format
    # and select only the rows corresponding to users in pred_df
    _user_tensor = self.sparse.to_dense().index_select(
        dim=0, index=torch.LongTensor(pred_df['userId'].reset_index(drop=True))
    )

    # Generate predictions using the self-expressiveness matrix (B)
    # _preds_tensor shape: [num_users x num_items]
    # Each row contains predicted scores for all items for a user
    _preds_tensor = _user_tensor @ self.B
    
    # If remove_owned is True, heavily penalize items the user has already interacted with
    # by subtracting a large value (-10.0) from their prediction scores
    if remove_owned:
        _preds_tensor += -10.0 * _user_tensor

    # For each user's predictions:
    # 1. Get indices of top-k items with highest scores using torch.topk()
    # 2. Convert to list and append to _output_preds
    for _preds in _preds_tensor:
        _output_preds.append(
            [_id for _id in _preds.topk(k).indices.tolist()]
        )

    # Add predictions as a new column to the input DataFrame
    pred_df["predicted_items"] = _output_preds
    return pred_df

In [49]:
ease = EASE(train, 2000)
preds_ease = ease.predict_all(df, 100)

preds_ease

,userId,predicted_items
0,0,"[19, 6, 141, 78, 32, 40, 143, 42, 16, 35, 50, ..."
13,1,"[19, 34, 50, 8, 43, 99, 16, 30, 78, 33, 143, 1..."
27,2,"[31, 16, 50, 34, 99, 65, 40, 17, 1, 58, 78, 63..."
42,3,"[23, 6, 16, 99, 50, 42, 3, 141, 30, 33, 78, 55..."
57,4,"[32, 16, 6, 30, 31, 65, 141, 143, 33, 78, 24, ..."
...,...,...
190558,11085,"[23, 19, 34, 42, 6, 203, 85, 43, 8, 30, 31, 3,..."
190574,11086,"[23, 32, 43, 6, 3, 66, 19, 78, 50, 99, 10, 17,..."
190585,11087,"[99, 34, 6, 65, 42, 78, 32, 16, 196, 3, 210, 8..."
190598,11088,"[19, 43, 32, 31, 6, 78, 8, 34, 30, 65, 16, 66,..."


In [50]:
import os
import pandas as pd
from caserec.recommenders.item_recommendation.userknn import UserKNN
from caserec.recommenders.item_recommendation.itemknn import ItemKNN
from caserec.recommenders.item_recommendation.item_attribute_knn import ItemAttributeKNN

## Funções auxiliares

In [51]:
def get_tags(movieId: int):
    """
    Retrieve all tags associated with a specific movie.
    
    Args:
        movieId (int): The ID of the movie to look up
        
    Returns:
        list: A list of tags for the movie. Returns empty list if movie not found.
        
    Note:
        Assumes existence of a global 'tags' DataFrame with columns 'movieId' and 'tag'
    """
    # Check if movie exists in the tags dataset
    if movieId not in tags['movieId'].values:
        return []
    # Return all tags associated with the movie as a list
    return tags.loc[(tags.movieId==movieId),'tag'].tolist()

def get_genres(movieId: int):
    """
    Retrieve all genres associated with a specific movie.
    
    Args:
        movieId (int): The ID of the movie to look up
        
    Returns:
        list: A list of genres for the movie. Returns empty list if movie not found
             or if genres are not available (NA value).
        
    Note:
        Assumes existence of a global 'movies' DataFrame with columns 'movieId' and 'genres'
        Genres are expected to be stored as pipe-separated strings (e.g., "Action|Adventure|Sci-Fi")
    """
    # Check if movie exists in the movies dataset
    if movieId not in movies['movieId'].values:
        return []  
    
    # Get the genre string for the movie
    genre_str = movies.loc[movies['movieId'] == movieId, 'genres'].iloc[0]
    # Split genre string by '|' if it exists, otherwise return empty list
    return genre_str.split('|') if pd.notna(genre_str) else []

def item_sim_jaccard(movieId1, movieId2, data_type='tag'):
    """
    Calculate Jaccard similarity between two movies based on their tags or genres.
    
    Jaccard similarity = size of intersection / size of union
    
    Args:
        movieId1 (int): ID of first movie
        movieId2 (int): ID of second movie
        data_type (str): Type of data to compare - either 'tag' or 'genre'
    
    Returns:
        float: Jaccard similarity score between 0 and 1
               0 = no overlap between tags/genres
               1 = identical tags/genres
    
    Raises:
        ValueError: If data_type is not 'tag' or 'genre'
    """
    # Get appropriate lists based on data type
    if data_type == 'tag':
        list1 = get_tags(movieId1)  
        list2 = get_tags(movieId2)
    elif data_type == 'genre':
        list1 = get_genres(movieId1)  
        list2 = get_genres(movieId2)
    else:
        raise ValueError("data_type must be 'tag' ir 'genre'")
    
    # Find common elements between the two lists
    common_items = list(set(list1) & set(list2))
    
    # If no common items, similarity is 0
    if len(common_items) == 0:
        return 0.0
    
    # Calculate Jaccard similarity:
    # Size of intersection divided by size of union
    return len(common_items) / len(set(list1 + list2))

def generate_sim_item_file(train: pd.DataFrame, method: str = 'tag'):
    """
    Generate a similarity matrix file for all unique items (movies) in the training data.
    The similarity is calculated using the Jaccard similarity coefficient based on either
    tags or genres.
    
    Args:
        train (pd.DataFrame): Training data containing movieId column
        method (str): Method to calculate similarity - either 'tag' or 'genre'
                     'tag': Uses movie tags for similarity calculation
                     'genre': Uses movie genres for similarity calculation
    
    Outputs:
        Creates a file 'sim_temp_matrix.dat' with format:
        movieId1    movieId2    similarity_score
        Each line represents the similarity between two movies.
    
    Raises:
        ValueError: If method is not 'tag' or 'genre'
    
    Note:
        - Only calculates upper triangular matrix to avoid redundancy
        - Assumes existence of item_sim_jaccard function
        - Writes similarities in tab-separated format
    """
    # Validate input method
    if method not in ['tag', 'genre']:
        raise ValueError("Method must be 'tag' or 'genre'")
    
    # Get unique movies from training data
    unique_items = train['movieId'].unique()
    
    # Open file in write mode to store similarity matrix
    with open('sim_temp_matrix.dat', 'w') as arq_sim_matrix:
        # Iterate through all unique pairs of movies
        # Using i_idx + 1 in second loop to only compute upper triangular matrix
        # This avoids redundant calculations since sim(i,j) = sim(j,i)
        for i_idx, i in enumerate(unique_items):
            for j in unique_items[i_idx + 1:]:
                # Calculate Jaccard similarity between movies i and j
                sim_score = item_sim_jaccard(i, j, method)
                # Write result to file in format: movie1 movie2 similarity
                arq_sim_matrix.write(f"{i}\t{j}\t{sim_score}\n")

## Modelos para comparação

In [52]:
class Base_Knn:
    """
    Base class for KNN-based recommendation algorithms.
    Handles common functionality like file management and recommendation generation.
    """
    def __init__(self, train: pd.DataFrame, test: pd.DataFrame, rank_length: int = 10):
        """
        Initialize base KNN recommender.
        
        Args:
            train (pd.DataFrame): Training data with user-item interactions
            test (pd.DataFrame): Test data for prediction
            rank_length (int): Number of recommendations to generate per user
        """
        self.train = train
        self.test = test
        self.rank_length = rank_length
        # Temporary file paths for external KNN computations
        self.train_file = 'train_temp.dat'
        self.test_file = 'test_temp.dat'
        self.output_file = 'rec_temp.dat'
    
    def save_temp_files(self):
        """Save training and test data to temporary files in tab-separated format."""
        self.train.to_csv(self.train_file, index=False, header=False, sep='\t')
        self.test.to_csv(self.test_file, index=False, header=False, sep='\t')
    
    def cleanup_temp_files(self, additional_files=None):
        """
        Remove temporary files after computation.
        
        Args:
            additional_files (list): Optional list of additional files to remove
        """
        temp_files = [self.train_file, self.test_file, self.output_file]
        if additional_files:
            temp_files.extend(additional_files)
        for file in temp_files:
            if os.path.exists(file):
                os.remove(file)
    
    def read_predictions(self):
        """
        Read and sort KNN predictions from output file.
        Returns sorted DataFrame with user, movie, and rating predictions.
        """
        pred_knn = pd.read_csv(self.output_file, sep='\t', names=['userId', 'movieId', 'relative_rating'])
        pred_knn = pred_knn.sort_values(by=['userId', 'relative_rating'], ascending=[True, False])
        return pred_knn

    def generate_recommendations(self, pred_knn, pred_df):
        """
        Generate final recommendations list for each user.
        
        Args:
            pred_knn (pd.DataFrame): Raw predictions from KNN
            pred_df (pd.DataFrame): DataFrame with user IDs to generate recommendations for
            
        Returns:
            pd.DataFrame: DataFrame with user IDs and their recommended items
        """
        pred_items = pred_knn.groupby('userId')['movieId'].apply(list).reset_index()
        pred_items.rename(columns={'movieId': 'predicted_items'}, inplace=True)
        pred_df = pred_df[['userId']].drop_duplicates()
        pred_df = pred_df.merge(pred_items, on='userId', how='left')
        return pred_df

class User_Knn(Base_Knn):
    """
    User-based KNN recommender system.
    Finds similar users based on their rating patterns.
    """
    def __init__(self, train: pd.DataFrame, test: pd.DataFrame, similarity_metric: str = 'cosine', rank_length: int = 10):
        """
        Args:
            similarity_metric (str): Method to compute user similarity (e.g., 'cosine')
        """
        super().__init__(train, test, rank_length)
        self.similarity_metric = similarity_metric

    def predict_all(self, pred_df: pd.DataFrame):
        """
        Generate recommendations using user-based KNN approach.
        """
        self.save_temp_files()
        UserKNN(train_file=self.train_file, test_file=self.test_file, output_file=self.output_file,
                similarity_metric=self.similarity_metric, rank_length=self.rank_length
                ).compute(verbose=True, verbose_evaluation=True)
        pred_knn = self.read_predictions()
        result = self.generate_recommendations(pred_knn, pred_df)
        self.cleanup_temp_files()
        return result

class Item_Knn(Base_Knn):
    """
    Item-based KNN recommender system.
    Finds similar items based on user rating patterns.
    """
    def __init__(self, train: pd.DataFrame, test: pd.DataFrame, similarity_metric: str = 'cosine', rank_length: int = 10):
        """
        Args:
            similarity_metric (str): Method to compute item similarity (e.g., 'cosine')
        """
        super().__init__(train, test, rank_length)
        self.similarity_metric = similarity_metric

    def predict_all(self, pred_df: pd.DataFrame):
        """
        Generate recommendations using item-based KNN approach.
        """
        self.save_temp_files()
        ItemKNN(train_file=self.train_file, test_file=self.test_file, output_file=self.output_file,
                similarity_metric=self.similarity_metric, rank_length=self.rank_length
                ).compute(verbose=True, verbose_evaluation=True)
        pred_knn = self.read_predictions()
        result = self.generate_recommendations(pred_knn, pred_df)
        self.cleanup_temp_files()
        return result

class ItemAttribute_Knn(Base_Knn):
    """
    Item-attribute based KNN recommender system.
    Finds similar items based on specific item attributes (e.g., genre, tags).
    """
    def __init__(self, train: pd.DataFrame, test: pd.DataFrame, attribute: str, rank_length: int = 10):
        """
        Args:
            attribute (str): Item attribute to use for similarity computation
        """
        super().__init__(train, test, rank_length)
        self.attribute = attribute
        self.similarity_file = 'sim_temp_matrix.dat'

    def predict_all(self, pred_df: pd.DataFrame):
        """
        Generate recommendations using item-attribute based KNN approach.
        Uses external similarity matrix generation and ItemAttributeKNN implementation.
        """
        self.save_temp_files()
        # Generate similarity matrix based on specified attribute
        generate_sim_item_file(train=self.train, method=self.attribute)
        ItemAttributeKNN(train_file=self.train_file, test_file=self.test_file, output_file=self.output_file,
                         similarity_file=self.similarity_file, rank_length=self.rank_length
                         ).compute(verbose=True, verbose_evaluation=True)
        pred_knn = self.read_predictions()
        result = self.generate_recommendations(pred_knn, pred_df)
        self.cleanup_temp_files(additional_files=[self.similarity_file])
        return result

userknn = User_Knn(train = train[['userId', 'movieId', 'rating']], test = test[['userId', 'movieId', 'rating']], rank_length = 100).predict_all(df)
itemknn = Item_Knn(train = train[['userId', 'movieId', 'rating']], test = test[['userId', 'movieId', 'rating']], rank_length = 100).predict_all(df)
itemattribute_tag = ItemAttribute_Knn(train = train[['userId', 'movieId', 'rating']], test = test[['userId', 'movieId', 'rating']], attribute = 'tag', rank_length = 100).predict_all(df)
itemattribute_genre = ItemAttribute_Knn(train = train[['userId', 'movieId', 'rating']], test = test[['userId', 'movieId', 'rating']], attribute = 'genre', rank_length = 100).predict_all(df)

[Case Recommender: Item Recommendation > UserKNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 11.534787 sec
prediction_time:: 159.539348 sec


Eval:: PREC@1: 0.445559 PREC@3: 0.319774 PREC@5: 0.263986 PREC@10: 0.190871 RECALL@1: 0.147543 RECALL@3: 0.297132 RECALL@5: 0.399217 RECALL@10: 0.562056 MAP@1: 0.445559 MAP@3: 0.537784 MAP@5: 0.53791 MAP@10: 0.505599 NDCG@1: 0.445559 NDCG@3: 0.624655 NDCG@5: 0.638181 NDCG@10: 0.627185 
[Case Recommender: Item Recommendation > ItemKNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 0.549309 sec
prediction_time:: 36.526676 sec


Eval:: PREC@1: 0.422193 PREC@3: 0.307003 PREC@5: 0.252521 PREC@10: 0.187201 RECALL@1: 0.138504 RECALL@3: 0.285647 RECALL@5: 0.382874 RECALL@1

In [40]:
userknn

,userId,predicted_items
0,0,"[507, 1211, 1158, 706, 902, 398, 958, 615, 964..."
1,1,"[2226, 6315, 4137, 3638, 314, 4800, 461, 510, ..."
2,2,"[224, 915, 898, 510, 911, 418, 507, 900, 1939,..."
3,3,"[2250, 1939, 2195, 900, 224, 836, 659, 964, 96..."
4,4,"[277, 509, 395, 314, 418, 123, 138, 334, 249, ..."
...,...,...
605,605,"[277, 3141, 2982, 1267, 2302, 2145, 1398, 1071..."
606,606,"[900, 2078, 902, 0, 546, 510, 314, 1067, 334, ..."
607,607,"[615, 1158, 4800, 2078, 863, 224, 1734, 314, 9..."
608,608,"[307, 337, 302, 138, 126, 43, 506, 510, 253, 2..."


In [41]:
itemknn

,userId,predicted_items
0,0,"[902, 1158, 793, 1211, 964, 1979, 2078, 507, 5..."
1,1,"[8063, 8990, 8372, 7090, 6331, 6534, 8879, 674..."
2,2,"[681, 2764, 2248, 1273, 786, 2636, 789, 2978, ..."
3,3,"[520, 659, 224, 828, 923, 705, 907, 964, 900, ..."
4,4,"[509, 123, 505, 334, 436, 418, 314, 156, 472, ..."
...,...,...
605,605,"[898, 2145, 3141, 964, 1267, 4360, 1298, 3568,..."
606,606,"[900, 1067, 902, 969, 334, 0, 899, 314, 2078, ..."
607,607,"[1158, 2078, 939, 2038, 314, 224, 337, 418, 86..."
608,608,"[337, 126, 138, 307, 302, 334, 253, 275, 322, ..."


In [13]:
itemattribute_tag

,userId,predicted_items
0,0,"[19, 1, 141, 6, 58, 42, 33, 126, 40, 3, 129, 1..."
1,1,"[19, 141, 1, 37, 273, 126, 196, 35, 33, 58, 12..."
2,2,"[1, 141, 31, 126, 223, 17, 85, 58, 51, 196, 17..."
3,3,"[1, 37, 141, 6, 273, 196, 126, 23, 7, 123, 85,..."
4,4,"[37, 32, 1, 126, 141, 196, 33, 223, 65, 35, 44..."
...,...,...
11085,11085,"[42, 19, 1, 196, 35, 23, 37, 8, 6, 123, 24, 64..."
11086,11086,"[32, 19, 141, 37, 223, 273, 17, 6, 126, 64, 85..."
11087,11087,"[1, 37, 196, 8, 33, 171, 126, 65, 32, 223, 42,..."
11088,11088,"[1, 141, 120, 32, 42, 115, 19, 273, 33, 65, 24..."


In [14]:
itemattribute_genre

,userId,predicted_items
0,0,"[129, 141, 35, 126, 62, 1, 4, 20, 86, 109, 160..."
1,1,"[37, 64, 109, 19, 126, 223, 62, 127, 35, 53, 1..."
2,2,"[17, 127, 62, 115, 223, 1, 4, 20, 86, 89, 126,..."
3,3,"[1, 4, 20, 86, 160, 171, 17, 127, 37, 126, 42,..."
4,4,"[32, 62, 1, 4, 20, 86, 126, 160, 171, 196, 123..."
...,...,...
11085,11085,"[127, 17, 58, 42, 8, 30, 6, 37, 22, 23, 25, 51..."
11086,11086,"[32, 127, 17, 244, 247, 223, 37, 66, 89, 181, ..."
11087,11087,"[8, 30, 127, 53, 17, 32, 44, 125, 181, 244, 58..."
11088,11088,"[109, 273, 53, 62, 6, 51, 52, 171, 235, 58, 14..."


## Métricas de desempenho

In [15]:
def average_precision(user: int, test: pd.DataFrame, user_recommendations: list, limit: int) -> float:
    """
    Calculate Average Precision (AP) for a single user's recommendations.
    AP measures the average of precision values at each relevant item in the ranked list.
    
    Args:
        user (int): User ID to evaluate
        test (pd.DataFrame): Ground truth data containing actual user-item interactions
        user_recommendations (list): Ordered list of recommended items for the user
        limit (int): Number of recommendations to consider (k)
    
    Returns:
        float: Average Precision score (0-1)
    """
    # Get actual items the user interacted with (ground truth)
    ground_truth = test.loc[(test.userId==user), 'movieId'].tolist()
    total = 0  # Number of items examined
    hits = 0   # Number of relevant items found
    ap = 0.0   # Running sum of precision values
    
    # Calculate precision at each position in recommendations
    for val in user_recommendations:
        total += 1
        if val in ground_truth:  # If recommendation is relevant
            hits += 1
            ap += hits / total   # Add precision at this position
        if total == limit:       # Stop after examining k items
            break
            
    # Normalize by number of hits (relevant items found)
    if hits:
        ap /= hits
    return ap

In [16]:
def mean_average_precision(test: pd.DataFrame, recommendations: pd.DataFrame, limit: int = 1) -> float:
    """
    Calculate Mean Average Precision (MAP) across all users.
    MAP is the mean of Average Precision scores for all users.
    
    Args:
        test (pd.DataFrame): Ground truth data
        recommendations (pd.DataFrame): DataFrame with user recommendations
        limit (int): Number of recommendations to consider (k)
    
    Returns:
        float: MAP score (0-1)
    """
    mean_avg_precision = 0.0
    users = list(map_users.values())
    
    # Calculate AP for each user and sum
    for user in users:
        mean_avg_precision += average_precision(
            user, 
            test, 
            recommendations.loc[(recommendations.userId == user), 'predicted_items'].values[0], 
            limit
        )
    
    # Return average across all users
    return mean_avg_precision / len(users)


In [17]:
def recall(user: int, test: pd.DataFrame, user_recommendations: list, limit: int) -> float:
    """
    Calculate Recall for a single user's recommendations.
    Recall measures the fraction of relevant items that are successfully retrieved.
    
    Args:
        user (int): User ID to evaluate
        test (pd.DataFrame): Ground truth data
        user_recommendations (list): Ordered list of recommended items
        limit (int): Number of recommendations to consider (k)
    
    Returns:
        float: Recall score (0-1)
    """
    # Get ground truth items
    ground_truth = test.loc[(test.userId==user), 'movieId'].tolist()
    
    # Find all relevant items in recommendations
    relevant = list(set(user_recommendations) & set(ground_truth))
    if len(relevant) == 0:  # If user has no relevant items
        return 0.0
    
    # Find relevant items in top-k recommendations
    relevant_k = list(set(user_recommendations[:limit]) & set(ground_truth))
    
    # Return fraction of all relevant items found in top-k
    return len(relevant_k) / len(ground_truth)

In [18]:
def mean_recall(test: pd.DataFrame, recommendations: pd.DataFrame, limit: int = 1) -> float:
    """
    Calculate Mean Recall across all users.
    
    Args:
        test (pd.DataFrame): Ground truth data
        recommendations (pd.DataFrame): DataFrame with user recommendations
        limit (int): Number of recommendations to consider (k)
    
    Returns:
        float: Mean Recall score (0-1)
    """
    mean_recall = 0.0
    users = list(map_users.values())
    
    # Calculate recall for each user and sum
    for user in users:
        mean_recall += recall(
            user, 
            test, 
            recommendations.loc[(recommendations.userId == user), 'predicted_items'].values[0], 
            limit
        )
    
    # Return average across all users
    return mean_recall / len(users)

In [19]:
def precision(user: int, test: pd.DataFrame, user_recommendations: list, limit: int) -> float:
    """
    Calculate Precision for a single user's recommendations.
    Precision measures the fraction of recommended items that are relevant.
    
    Args:
        user (int): User ID to evaluate
        test (pd.DataFrame): Ground truth data
        user_recommendations (list): Ordered list of recommended items
        limit (int): Number of recommendations to consider (k)
    
    Returns:
        float: Precision score (0-1)
    """
    ground_truth = test.loc[(test.userId==user), 'movieId'].tolist()
    total = 0  # Number of recommendations examined
    hits = 0   # Number of relevant recommendations
    
    # Count relevant items in top-k recommendations
    for val in user_recommendations:
        total += 1
        if val in ground_truth:
            hits += 1
        if total == limit:
            break
            
    return hits/total

In [20]:
def mean_precision(test: pd.DataFrame, recommendations: pd.DataFrame, limit: int = 1) -> float:
    """
    Calculate Mean Precision across all users.
    
    Args:
        test (pd.DataFrame): Ground truth data
        recommendations (pd.DataFrame): DataFrame with user recommendations
        limit (int): Number of recommendations to consider (k)
    
    Returns:
        float: Mean Precision score (0-1)
    """
    mean_precision = 0.0
    users = list(map_users.values())
    
    # Calculate precision for each user and sum
    for user in users:
        mean_precision += precision(
            user, 
            test, 
            recommendations.loc[(recommendations.userId == user), 'predicted_items'].values[0], 
            limit
        )
    
    # Return average across all users
    return mean_precision / len(users)

In [21]:
def ndcg(user: int, test: pd.DataFrame, user_recommendations: list, limit: int) -> float:
    """
    Calculate Normalized Discounted Cumulative Gain (NDCG) for a single user's recommendations.
    NDCG measures the quality of ranking by incorporating position-dependent weights.
    
    Args:
        user (int): User ID to evaluate
        test (pd.DataFrame): Ground truth data containing actual user-item interactions
        user_recommendations (list): Ordered list of recommended items for the user
        limit (int): Number of recommendations to consider (k)
    
    Returns:
        float: NDCG score (0-1)
        - 0 means poor ranking quality
        - 1 means perfect ranking
    """
    # Get actual items the user interacted with (ground truth)
    ground_truth = test.loc[(test.userId==user), 'movieId'].tolist()
    
    total = 0  # Number of items examined
    hits = 0   # Number of relevant items found
    dcg = 0.0  # Discounted Cumulative Gain
    
    # Calculate DCG
    for val in user_recommendations:
        total += 1
        if val in ground_truth:
            hits += 1
            # Add position-discounted gain
            # Position is (total + 1) because log2(1) = 0
            dcg += 1 / (np.log2(total + 1))
        if total == limit:
            break
    
    # If no relevant items found, NDCG is 0
    if hits == 0:
        return 0.0
        
    # Calculate Ideal DCG (IDCG)
    # IDCG is DCG score for perfect ranking where all relevant items appear first
    idcg = 0.0
    for i in range(hits):
        # i+2 because position starts at 1 and we need log2(position)
        idcg += 1 / (np.log2(i+2))
    
    # Return normalized score (DCG/IDCG)
    return dcg / idcg

In [22]:
def mean_ndcg(test: pd.DataFrame, recommendations: pd.DataFrame, limit: int = 1) -> float:
    """
    Calculate Mean NDCG across all users.
    
    Args:
        test (pd.DataFrame): Ground truth data
        recommendations (pd.DataFrame): DataFrame with user recommendations
        limit (int): Number of recommendations to consider (k)
    
    Returns:
        float: Mean NDCG score (0-1)
    """
    mean_ndcg = 0.0
    users = list(map_users.values())
    
    # Calculate NDCG for each user and sum
    for user in users:
        mean_ndcg += ndcg(
            user, 
            test, 
            recommendations.loc[(recommendations.userId == user), 'predicted_items'].values[0], 
            limit
        )
    
    # Return average across all users
    return mean_ndcg / len(users)

In [23]:
preds_ease

,userId,predicted_items
0,0,"[19, 6, 141, 40, 78, 32, 143, 42, 35, 203, 50,..."
13,1,"[19, 50, 34, 8, 43, 30, 99, 16, 33, 78, 143, 1..."
27,2,"[31, 16, 50, 34, 99, 65, 40, 1, 17, 63, 58, 4,..."
42,3,"[23, 6, 16, 99, 50, 42, 3, 141, 33, 55, 30, 37..."
57,4,"[32, 16, 6, 30, 31, 65, 33, 141, 143, 78, 40, ..."
...,...,...
190558,11085,"[23, 19, 34, 6, 42, 203, 85, 8, 43, 30, 31, 66..."
190574,11086,"[23, 32, 43, 6, 66, 3, 78, 19, 50, 99, 10, 17,..."
190585,11087,"[99, 34, 6, 42, 65, 78, 32, 16, 210, 196, 3, 2..."
190598,11088,"[43, 19, 32, 31, 6, 78, 8, 65, 30, 34, 66, 10,..."


In [ ]:
# Dictionary mapping recommendation method names to their prediction DataFrames
methods = {
    "EASE": preds_ease,
    "user_knn": userknn,
    "item_knn": itemknn,
    "itemattribute_tag": itemattribute_tag,
    "itemattribute_genre": itemattribute_genre
}
    
# List to store evaluation metrics for each recommendation method
results = []

# Iterate through each recommendation method and its predictions
for method_name, recommendation_df in methods.items():
    # Calculate Mean Average Precision at K=10
    map_score = mean_average_precision(test, recommendation_df, 10)
    # Calculate Recall at K=10
    recall_score = mean_recall(test, recommendation_df, 10)
    # Calculate Precision at K=10
    precision_score = mean_precision(test, recommendation_df, 10)
    # Calculate Normalized Discounted Cumulative Gain at K=10
    ndcg_score = mean_ndcg(test, recommendation_df, 10)

    # Store results for current method in a dictionary
    results.append({
        "method": method_name,
        "MAP@10": map_score,
        "RECALL@10": recall_score,
        "PRECISION@10": precision_score,
        "NDCG@10": ndcg_score
    })

# Convert results list to a pandas DataFrame for easier comparison
results_df = pd.DataFrame(results)
# Display the results DataFrame showing performance metrics for all methods
results_df

,method,MAP@10,RECALL@10,PRECISION@10,NDCG@10
0,EASE,0.507238,0.172833,0.298689,0.614132
1,user_knn,0.446242,0.165572,0.266393,0.566522
2,item_knn,0.484428,0.158720,0.284262,0.595014
